In [1]:
# Mount google drive

from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!rm -r data/
!rm -r __MACOSX/
!rm -r test/
!rm -r train/
!rm -r valid/
!rm data.*
!rm README.*

rm: cannot remove 'data/': No such file or directory
rm: cannot remove '__MACOSX/': No such file or directory
rm: cannot remove 'test/': No such file or directory
rm: cannot remove 'train/': No such file or directory
rm: cannot remove 'valid/': No such file or directory
rm: cannot remove 'data.*': No such file or directory
rm: cannot remove 'README.*': No such file or directory


In [3]:
!cp -r drive/MyDrive/CS4487/test.zip ./
!cp drive/MyDrive/CS4487/Swin_*.pth ./
!unzip test.zip

串流輸出內容已截斷至最後 5000 行。
  inflating: test/image_0002064.jpg  
  inflating: __MACOSX/test/._image_0002064.jpg  
  inflating: test/image_0000673.jpg  
  inflating: __MACOSX/test/._image_0000673.jpg  
  inflating: test/image_0000115.jpg  
  inflating: __MACOSX/test/._image_0000115.jpg  
  inflating: test/image_0000101.jpg  
  inflating: __MACOSX/test/._image_0000101.jpg  
  inflating: test/image_0001579.jpg  
  inflating: __MACOSX/test/._image_0001579.jpg  
  inflating: test/image_0000667.jpg  
  inflating: __MACOSX/test/._image_0000667.jpg  
  inflating: test/image_0002070.jpg  
  inflating: __MACOSX/test/._image_0002070.jpg  
  inflating: test/image_0001551.jpg  
  inflating: __MACOSX/test/._image_0001551.jpg  
  inflating: test/image_0000897.jpg  
  inflating: __MACOSX/test/._image_0000897.jpg  
  inflating: test/image_0002058.jpg  
  inflating: __MACOSX/test/._image_0002058.jpg  
  inflating: test/image_0000129.jpg  
  inflating: __MACOSX/test/._image_0000129.jpg  
  inflating: test/imag

In [4]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
# ===============================
# Test Dataset
# ===============================
class TestDataset(Dataset):
    def __init__(self, test_dir, csv_path=None):
        self.test_dir = test_dir

        if csv_path is not None:
            # Validation mode (with labels)
            self.data = pd.read_csv(csv_path, dtype={'ID': str})
            self.has_labels = True
        else:
            # Submission mode (no labels)
            img_files = [f for f in os.listdir(test_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            self.data = pd.DataFrame({'ID': [os.path.splitext(f)[0] for f in img_files]})
            self.has_labels = False

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_id = str(self.data.iloc[idx]['ID'])
        img_path = os.path.join(self.test_dir, img_id + ".jpg")

        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)

        if self.has_labels:
            label = int(self.data.iloc[idx]['label'])
            return image, label, img_id
        else:
            return image, img_id

In [6]:
# ===============================
# CNN Model
# ===============================
import timm

class CNN(nn.Module):
    def __init__(self, pretrained=True, freeze_backbone=True, dropout=0.3):
        super(CNN, self).__init__()
        # Swin-Base
        self.swin = timm.create_model('swin_base_patch4_window7_224', pretrained=pretrained, num_classes=0)  # 1024-dim

        # Freeze backbones (highly recommended for AIGC detection)
        if freeze_backbone:
            for p in self.swin.parameters():
                p.requires_grad = False

        # Fusion head
        self.fusion = nn.Sequential(
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),

            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),

            nn.Linear(128, 2)  # ← 2 classes: real vs AIGC
        )

    def forward(self, x):
        swin_feat = self.swin(x)

        if swin_feat.dim() == 3:
            swin_feat = swin_feat.mean(1)

        out = self.fusion(swin_feat)
        return out

In [7]:
def test_model():
    data_root = "./"
    model_paths = [x for x in os.listdir() if x.endswith(".pth")]
    for model_path in model_paths:
        # model_path = "model.pth"

        test_dir = os.path.join(data_root, "test")
        output_csv = f"submission_{model_path[:-4]}.csv"

        print(f"Loading images from: {test_dir}")
        print(f"Loading model from: {model_path}")

        # Load Dataset
        test_dataset = TestDataset(test_dir, csv_path=None)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0, pin_memory=True)

        # Load model
        model = CNN(pretrained=False, freeze_backbone=True).to(device)
        # Load trained weights from the model
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.eval()
        print("Model loaded")

        results = []

        with torch.no_grad():
            for batch in tqdm(test_loader, desc="Predicting"):
                if len(batch) == 3:
                    imgs, _, img_ids = batch
                else:
                    imgs, img_ids = batch

                imgs = imgs.to(device)
                outputs = model(imgs)
                preds = outputs.argmax(dim=1).cpu().numpy()  # 0 or 1

                for img_id, pred in zip(img_ids, preds):
                    results.append({"ID": img_id, "label": int(pred)})

        # Save to CSV
        submission_df = pd.DataFrame(results)
        submission_df.to_csv(output_csv, index=False)
        print(f"\nSubmission saved to '{output_csv}'")
        print(f"Total predictions: {len(submission_df)}")
        print(submission_df.head())

# Run it
if __name__ == "__main__":
    test_model()

Loading images from: ./test
Loading model from: Swin_10e.pth
Model loaded


Predicting: 100%|██████████| 79/79 [00:26<00:00,  2.99it/s]



Submission saved to 'submission_Swin_10e.csv'
Total predictions: 2500
              ID  label
0  image_0001350      1
1  image_0001564      0
2  image_0001376      1
3  image_0000227      1
4  image_0001098      1
Loading images from: ./test
Loading model from: Swin_5e.pth
Model loaded


Predicting: 100%|██████████| 79/79 [00:25<00:00,  3.09it/s]



Submission saved to 'submission_Swin_5e.csv'
Total predictions: 2500
              ID  label
0  image_0001350      1
1  image_0001564      0
2  image_0001376      1
3  image_0000227      1
4  image_0001098      1
Loading images from: ./test
Loading model from: Swin_15e.pth
Model loaded


Predicting: 100%|██████████| 79/79 [00:25<00:00,  3.06it/s]


Submission saved to 'submission_Swin_15e.csv'
Total predictions: 2500
              ID  label
0  image_0001350      1
1  image_0001564      0
2  image_0001376      1
3  image_0000227      1
4  image_0001098      1


In [8]:
!cp submission_*.csv drive/MyDrive/CS4487/

In [ ]:
data_root = "path_to_dataset"
model_path = "saved_model.pth"